# Plot points on a map
```sql
-- get rental start and end lat/lngs
select 
  carsharing_rental.id,
  carsharing_rental.customer_id,
  carsharing_rental.created_at,
  s.lat as start_lat,
  s.lng as start_lng,
  e.lat as end_lat,
  e.lng as end_lng
from
  carsharing_rental
join common_location s on (s.id = carsharing_rental.start_location_id)
join common_location e on (e.id = carsharing_rental.end_location_id )
where
  [carsharing_rental.created_at=daterange:hpst] and
  carsharing_rental.end_datetime is not null
```

In [1]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON
import pandas as pd
from pyproj import Proj, transform


Use `output_notebook` if you want to use Bokeh in Jupyter Notebook

In [2]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
data_filename = 'Rentalstartandendlatlngs_2019-4-19_1414.csv'

df = pd.read_csv(data_filename, 
                 parse_dates=[2],
                 infer_datetime_format=True)
df.head()

,id,customer_id,created_at,start_lat,start_lng,end_lat,end_lng
0,599213,59802,2019-04-19 07:10:02.102230,38.569490,-121.47623,38.569473,-121.476180
1,599208,56614,2019-04-19 07:02:51.691177,37.867880,-122.25048,37.867910,-122.250510
2,599211,16102,2019-04-19 07:06:49.435180,37.866623,-122.25645,37.870773,-122.270900
3,599210,14162,2019-04-19 07:04:00.862194,37.875640,-122.25876,37.864952,-122.259346
4,599216,58730,2019-04-19 07:14:35.241074,38.580128,-121.50273,38.585570,-121.486460


Transform the rental start coordinates from WGS 84 (EPSG:4326) to Web Mercator (EPSG:3857) (because the tileset we will map on uses Web Mercator)

In [4]:
y2, x2 = transform(
    Proj(init='epsg:4326'), 
    Proj(init='epsg:3857'), 
    df['start_lng'].values, 
    df['start_lat'].values
)  # longitude first, latitude second.

Create the map.  The tileset from the CartoDB Tile Service uses *Web Mercator* projection

In [13]:
#  make the initial bounds of the map the same as the bounds of the data
p = figure(
        x_range=(min(y2), max(y2)), 
        y_range=(min(x2), max(x2)),
        x_axis_type="mercator", 
        y_axis_type="mercator",
        match_aspect=True,  # make sure the map isn't skewed?
)
p.add_tile(CARTODBPOSITRON)
show(p)

Plot circles at each point

In [14]:
source = ColumnDataSource(data=dict(lat=x2, lon=y2))
p.circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, source=source)

show(p)